In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.predict_functional import predict_functional
import numpy as np
from scipy.optimize import curve_fit
import random
from scipy.optimize import minimize, rosen, rosen_der
from scipy.integrate import odeint
import sys
import os

In [ ]:
## set working directory

os.chdir("\\Users\\madel\\Documents\\Grad School\\Lab\\MCFA Project\\Monospecies Growth Curves\\   ")
os.getcwd()

In [ ]:
## import data
## use Plate Layout Template

gcdf = pd.read_csv("\\Users\\madel\\Documents\\Grad School\\Lab\\MCFA Project\\Monospecies Growth Curves\\   \\   _Plate Layout.csv")
gcdf.head()

## if the data needs to be cleaned, clean into workingdf

workingdf = gcdf 
workingdf.head()

In [ ]:
## curve optimization code from SEH ##
## non-lag phase logistic model ###

def dX_dt(x_vector, t, u, K):
    new_x_vector = np.zeros(len(x_vector))
    #u is max growth rate rate; K is carrying capacity
    for i in range(0,len(x_vector)):
        new_x_vector[i] = (x_vector[i] * u) - (x_vector[i] * x_vector[i] * (u/K))
    return new_x_vector
    
# ODEINT SIMULATION
def simulate(X0,T,p):
    u, K = p
    X = odeint(dX_dt, X0, T, args=(u, K))
    return X

# OBJECTIVE FUNCTION
def objective(p):
    # simulate model
    u, K = p
    X = simulate(X0_VECTOR,TIME,(u,K))
    X_transpose = np.transpose(X)
    # calculate objective, sum of average MSE of each antibiotic conc growth curve
    obj = 0.0
    for i in range(len(X_transpose)):
        obj += np.mean((np.array(EXP_YDATA_VECTOR[i]) - np.array(X_transpose[i]))**2)
    # return result
    return obj

def run_optimization():
    # OPTIMIZE
    print ("Optimizing with 5 different initial parameter guesses...\n\tMSE:")
    p0s = [(random.uniform(0.05,1.5),random.uniform(0.05,1.5)) for i in range(0,5)]
    best_sol = [0,0]
    bestmse = 10000000000
    bnds = [(0.01,5),(0.01, 5)]
    for j in range(0,5): 
        solution = minimize(objective,p0s[j] ,method='SLSQP', bounds=bnds)
        p_optimized = solution.x
        mse = objective(p_optimized) 
        print ('\t'+str(round(mse,4)))
        if mse < bestmse or bestmse == 10000000000 and not np.isnan(mse):
            best_sol = solution
            bestmse = mse
    try:
        print ("Best [u, K] found: "+str(best_sol.x)+"\nMSE: "+str(bestmse))
        #Plot simulation optimized param compared to data
        simulation_ydata_vector_opt = simulate(X0_VECTOR, TIME, best_sol.x)
        simulation_ydata_vector_opt = np.transpose(simulation_ydata_vector_opt)
        for i in range(0,len(EXP_YDATA_VECTOR)):
            plt.plot(TIME, EXP_YDATA_VECTOR[i],'b.',label='data')
            plt.plot(TIME, simulation_ydata_vector_opt[i], color='red',label='model')
    except AttributeError:
        print ("Error caught")
    
    return best_sol, bestmse

# example with data
# time = each measurement cycle
# EXP_YDATA_VECTOR = list of three lists containing measurements, these are replicates
# X0_VECTOR = t=0 timpoint

#global variables
TIME = [0.0, 1.0044444444444445, 2.01, 3.015833333333333, 4.018888888888889, 5.020277777777777, 6.020833333333333, 7.021388888888889, 8.021666666666667, 9.022777777777778, 10.022777777777778, 11.025555555555556, 12.0275, 13.031666666666666, 14.036111111111111, 15.038611111111111, 16.038888888888888, 17.038888888888888, 18.04277777777778, 19.04611111111111, 20.046666666666667, 21.046666666666667, 22.71611111111111, 23.718333333333334, 24.720833333333335, 25.723333333333333, 26.726111111111113, 27.729444444444443, 29.73361111111111, 30.736944444444443, 31.739722222222223, 32.74305555555556, 39.471666666666664, 40.47222222222222, 42.4725, 43.47277777777778, 44.475, 45.47805555555556, 47.350833333333334, 48.31777777777778, 49.319722222222225, 50.32, 51.319722222222225, 52.320277777777775, 53.320277777777775, 54.320277777777775, 55.320277777777775, 56.32138888888889, 57.32138888888889, 58.32138888888889, 59.321666666666665]
EXP_YDATA_VECTOR = [[0.0413, 0.0456, 0.05149999999999999, 0.05679999999999999, 0.0653, 0.07339999999999999, 0.08329999999999999, 0.09489999999999998, 0.1076, 0.1192, 0.13179999999999997, 0.14889999999999998, 0.1634, 0.1763, 0.19440000000000002, 0.2147, 0.2323, 0.2525, 0.2818, 0.3026, 0.3206, 0.347, 0.3821, 0.391, 0.41079999999999994, 0.42900000000000005, 0.43910000000000005, 0.4464, 0.4609, 0.4635, 0.47450000000000003, 0.4767, 0.4776, 0.48460000000000003, 0.4806, 0.48639999999999994, 0.48140000000000005, 0.4809, 0.48240000000000005, 0.48829999999999996, 0.4818, 0.479, 0.47719999999999996, 0.47450000000000003, 0.47540000000000004, 0.47840000000000005, 0.483, 0.48240000000000005, 0.48019999999999996, 0.4748, 0.47719999999999996], [0.04371516777727594, 0.04793333351855269, 0.08379736991761855, 0.06679780108553651, 0.0645338326464647, 0.03560106057991988, 0.08269302379452816, 0.13153995191944187, 0.1296812016267298, 0.15007024166142152, 0.12477862502549653, 0.13443117044368302, 0.1639908437147125, 0.17901038165997163, 0.22898312276387162, 0.22782735966199832, 0.24820089341611623, 0.2733464109913057, 0.3083034258823774, 0.3338817803244434, 0.33842125895754327, 0.3485297107577491, 0.35780054825560886, 0.4027586674420813, 0.4049270280070705, 0.4567939475742931, 0.4102114343122275, 0.47867268842236893, 0.4249370188383107, 0.46273962027266113, 0.4515686321010025, 0.4821240255062641, 0.49922795320954616, 0.49669693983785757, 0.44987805383589946, 0.5096885284198045, 0.4417530566434169, 0.4534393269454682, 0.5086957748719352, 0.5181930710433322, 0.47524271533356716, 0.5119432443677507, 0.4864830576834303, 0.4640442596818174, 0.47984316709793373, 0.4651890468416024, 0.45772949587906825, 0.49053300731865235, 0.4805804007416378, 0.5024813383606878, 0.4999375749830486],[0.046557427960620776, 0.03362236390439782, 0.059871526906901874, 0.04970494714613437, 0.054062676314341945, 0.06586258120214972, 0.07101622116924297, 0.0965109393724849, 0.1071059248468927, 0.10613621740938486, 0.14127644644100032, 0.14044764433521134, 0.17710766446632634, 0.16174848679356585, 0.19932664570676953, 0.2179272223227141, 0.24137551259577397, 0.26387805958555166, 0.26846082910659474, 0.3117533902685146, 0.31669370509426, 0.33214973940711834, 0.38077740849729547, 0.38818190025394883, 0.4007330540403484, 0.44266015144208215, 0.428911385959263, 0.4431160105113739, 0.4696620945515959, 0.4545911598992891, 0.46994557488387456, 0.4889619889432114, 0.48962244082657147, 0.4879399387001795, 0.4899940472762192, 0.4925893201453898, 0.4747730750262311, 0.4947992338978403, 0.47604567887229543, 0.4847087232155522, 0.4817948956075901, 0.48958103519820245, 0.4629030217483446, 0.47643105079786224, 0.4885942164698332, 0.47285725382189797, 0.49492014297156817, 0.49336021411480907, 0.48677272268507155, 0.47264344201513453, 0.4766579867150414]]#in  example 1 I am fitting 3 replicates, so only 3 Xn values followed by 3 Xg values
X0_VECTOR = [EXP_YDATA_VECTOR[0][0],EXP_YDATA_VECTOR[1][0],EXP_YDATA_VECTOR[2][0]]

#run optimization
p_optimized, mse = run_optimization()

In [ ]:
specieslist = [] # make a list of your species
medialist = [] # make a list of medias used
cyclecount = list(range(96)) # number of cycles, default is 96 (every 30min over 48 hours)
timepoints = ['0','12','24','36','48'] # for x axis ticks

In [ ]:
### Example ###
# species = Lb
# media = SCR
# number of reps = 3
# date = 10/28/20 # if working with data sheet that contains more than one day

## pull OD measurements from working df
eachdf = workingdf[(workingdf['Species']=='Lb')&(workingdf['Media']=='SCR')&(workingdf['Date']=='10.28.2020')]
indices = list(eachdf.index.values)
measurements = [[],[],[]] # list of measurements for each rep, here we are using 3 reps
tps = []
for cycle in cyclecount:
    for i in indices:
        if eachdf.at[i, 'Rep'] == 1:
            measurements[0].append(float(eachdf.at[i, str(cycle)]))
        if eachdf.at[i, 'Rep'] == 2:
            measurements[1].append(float(eachdf.at[i, str(cycle)]))
        if eachdf.at[i, 'Rep'] == 3:
            measurements[2].append(float(eachdf.at[i, str(cycle)]))
    tps.append(cycle)
## subtract minimum measurement, making sure none are exactly == 0
for k in range(len(measurements)): 
    minimum = min(measurements[k])
    for j in range(len(measurements[k])):
        measurements[k][j] = measurements[k][j] - (minimum - 0.0001) ## your smallest measurement will be 0.0001

## Define function vectors in terms of above data
TIME = tps
EXP_YDATA_VECTOR = measurements
X0_VECTOR = [EXP_YDATA_VECTOR[0][0],EXP_YDATA_VECTOR[1][0],EXP_YDATA_VECTOR[2][0]]

## create figure
fig, ax = plt.subplots(nrows=1, ncols=1, 
                    sharex = True, 
                    sharey = True, 
                    figsize=(10,10),
                    )
fig.suptitle(('Lb Optimized Curve SCR 10.28.2020'), fontsize = 14, x=0.5, y=0.9) # create figure level title

## run optimization
p_optimized, mse = run_optimization()
plt.annotate("Best[u,K,kg] found: "+str(p_optimized.x)+"\nMSE:"+str(mse), (0.4,0.95), xycoords='axes fraction')#+"\nMSE:"+str(bestmse))

## save and close
#plt.savefig('Lb Optimized Curve 10_28_2020', bbox_inches='tight')
plt.show()
plt.close()